In [1]:
# This notebook compares the dopamine signal aligned to movement during the CoT task to the
# dopamine signal evoked by the same high and low frequency sounds played while the mouse is freely moving.